In [1]:
import random
import time
import numpy as np
import math 
import cv2
import gymnasium as gym
from gymnasium import spaces
from stable_baselines3.common.env_checker import check_env
from stable_baselines3 import PPO 
from typing import Callable
import os
import carla
import sys
sys.path.append('/home/carla-simulator/PythonAPI/carla') #Adds the directory to the path so that modules
from agents.navigation.global_route_planner import GlobalRoutePlanner
#from agents.navigation.global_route_planner_dao import GlobalRoutePlannerDAO

2024-05-05 12:46:39.948785: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-05 12:46:40.018120: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-05 12:46:40.335164: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/gp-coopperc/anaconda3/envs/Tensor/lib/python3.9/site-packages/cv2/../../lib64:/usr/local/cud

In [2]:
SECONDS_PER_EPISODE = 25

N_CHANNELS = 3
HEIGHT = 250
WIDTH = 250


FIXED_DELTA_SECONDS = 0.2

SHOW_PREVIEW = True

In [3]:
class CarEnv(gym.Env):
    SHOW_CAM = SHOW_PREVIEW
    STEER_AMT = 1.0
    im_width = WIDTH
    im_height = HEIGHT
    front_camera = None
    CAMERA_POS_Z = 1.3 
    CAMERA_POS_X = 1.4
    PREFERRED_SPEED = 60
    SPEED_THRESHOLD = 2
    
    
    def __init__(self):
        super(CarEnv, self).__init__()
        
        self.action_space = spaces.MultiDiscrete([9])

        self.observation_space = spaces.Box(low=0.0, high=255.0,
                                            shape=(HEIGHT, WIDTH, N_CHANNELS), dtype=np.uint8)
        
        self.client = carla.Client("localhost", 2000)
        self.client.set_timeout(4.0)
        
        self.client.load_world('Town01')
        
        self.world = self.client.get_world()
        

        self.settings = self.world.get_settings()
        self.settings.fixed_delta_seconds = FIXED_DELTA_SECONDS
        self.settings.no_rendering_mode = True
        #self.settings.synchronous_mode = False
        self.world.apply_settings(self.settings)
        
        self.blueprint_library = self.world.get_blueprint_library()
        self.model_3 = self.blueprint_library.filter("model3")[0]
        
    def maintain_speed(self,s):
        
        if s >= self.PREFERRED_SPEED:
            return 0
        elif s < self.PREFERRED_SPEED - self.SPEED_THRESHOLD:
            return 0.7 # think of it as % of "full gas"
        else:
            return 0.3 # tweak this if the car is way over or under preferred speed 

    def reset(self, seed=None, options=None):
        self.collision_hist = []
        self.actor_list = []
        
        self.spawn_points = self.world.get_map().get_spawn_points()
        self.start_point = self.spawn_points[2]
        
        # Define point A and point B
        self.point_a = self.start_point
        self.point_b = carla.Location(x=107.175446, y=129.416809, z=0.003263)  # Destination point
        
        # Set starting point
        #self.point_a = carla.Location(x=299.399994, y=129.750000, z=0.299536)

        #self.transform = carla.Transform(carla.Location(x=299.399994, y=129.750000, z=0.299536))


        self.vehicle = None
        while self.vehicle is None:
            try:
                # connect
                self.vehicle = self.world.spawn_actor(self.model_3, self.point_a)
            except:
                pass
            
        self.actor_list.append(self.vehicle)
        self.initial_location = self.vehicle.get_location()
        
        self.sem_cam = self.blueprint_library.find('sensor.camera.semantic_segmentation')
        self.sem_cam.set_attribute("image_size_x", f"{self.im_width}")
        self.sem_cam.set_attribute("image_size_y", f"{self.im_height}")
        self.sem_cam.set_attribute("fov", f"90")

        camera_init_trans = carla.Transform(carla.Location(z=self.CAMERA_POS_Z,x=self.CAMERA_POS_X))
        self.sensor = self.world.spawn_actor(self.sem_cam, camera_init_trans, attach_to=self.vehicle)
        self.actor_list.append(self.sensor)
        self.sensor.listen(lambda data: self.process_img(data))

        self.vehicle.apply_control(carla.VehicleControl(throttle=0.0, brake=0.0))
        time.sleep(2) # Wait for the car to stabilize
        # showing camera at the spawn point
        if self.SHOW_CAM:
            cv2.namedWindow('Sem Camera',cv2.WINDOW_AUTOSIZE)
            cv2.imshow('Sem Camera', self.front_camera)
            cv2.waitKey(1)
        colsensor = self.blueprint_library.find("sensor.other.collision")
        self.colsensor = self.world.spawn_actor(colsensor, camera_init_trans, attach_to=self.vehicle)
        self.actor_list.append(self.colsensor)
        self.colsensor.listen(lambda event: self.collision_data(event))

        while self.front_camera is None:
            time.sleep(0.01) # Wait for the camera to initialize

        self.episode_start = time.time()

        self.step_counter = 0
        self.vehicle.apply_control(carla.VehicleControl(throttle=0.0, brake=0.0))
        
        observation = self.front_camera

        return observation.astype(np.uint8), {}
    
    def step(self, action):
        self.step_counter +=1
        steer = action[0]
        # map steering actions
        if steer == 0:
            steer = -0.9
        elif steer == 1:
            steer = -0.25
        elif steer == 2:
            steer = -0.1
        elif steer == 3:
            steer = -0.05
        elif steer == 4:
            steer = 0.0 
        elif steer == 5:
            steer = 0.05
        elif steer == 6:
            steer = 0.1
        elif steer == 7:
            steer = 0.25
        elif steer == 8:
            steer = 0.9


        v = self.vehicle.get_velocity()
        kmh = int(3.6 * math.sqrt(v.x**2 + v.y**2 + v.z**2))
        estimated_throttle = self.maintain_speed(kmh)

        self.vehicle.apply_control(carla.VehicleControl(throttle=estimated_throttle, steer=steer, brake = 0.0))

        #distance from the vehicle's current location to the destination point (point_b)
        distance_to_destination = self.vehicle.get_location().distance(self.point_b)

        # storing camera to return at the end in case the clean-up function destroys it
        cam = self.front_camera
        # showing image
        if self.SHOW_CAM:
            cv2.imshow('Sem Camera', cam)
            cv2.waitKey(1)

        # start defining reward from each step
        reward = 0
        done = False
        #punish for collision
        if len(self.collision_hist) != 0:
            done = True
            reward = reward - 300
            self.cleanup()
        #reward for acceleration
        if kmh < 15:
            reward = reward - 3
        elif kmh < 25:
            reward = reward - 2
        elif kmh < 50:
            reward = reward - 1
        elif kmh > 120:
            reward = reward - 10 #punish for going too fast
        else:
            reward = reward + 1
            
        # Calculate reward based on distance traveled towards point B    
        if distance_to_destination < 3:  # threshold for reaching destination
            reward = reward + 100  # Positive reward for reaching destination
            done = True  # End episode when destination is reached
        else:
            reward = reward - 0.05  # Negative reward for not reaching destination
            done = False

        # check for episode duration
        if self.episode_start + SECONDS_PER_EPISODE < time.time():
            done = True
            self.cleanup()
        
        # Modify observation to include distance to destination
        observation = np.append(self.front_camera, distance_to_destination)
        
        return observation.astype(np.uint8), reward, done, {}

    def cleanup(self):
        for sensor in self.world.get_actors().filter('*sensor*'):      
            sensor.destroy()
        for actor in self.world.get_actors().filter('*vehicle*'):
            actor.destroy()
        cv2.destroyAllWindows()
        
    def process_img(self, image):
        image.convert(carla.ColorConverter.CityScapesPalette)
        i = np.array(image.raw_data)
        i = i.reshape((self.im_height, self.im_width, 4))[:, :, :3] # this is to ignore the 4th Alpha channel - up to 3
        self.front_camera = i/255.0

    def collision_data(self, event):
        self.collision_hist.append(event)

In [4]:
env = CarEnv()

check_env(env, warn=True, skip_render_check = True) 

KeyboardInterrupt: 

In [4]:
models_dir = f"models/{int(time.time())}/"
logdir = f"logs/{int(time.time())}/"

if not os.path.exists(models_dir):
    os.makedirs(models_dir)

if not os.path.exists(logdir):
    os.makedirs(logdir)


env = CarEnv()

env.reset()

model = PPO('MlpPolicy', env, verbose=1,learning_rate=0.001,tensorboard_log=logdir)

TIMESTEPS = 500_000 # individual steps
iters = 0
while iters<4:  # how many training iterations you want
    iters += 1
    print('Iteration ', iters,' is to start...')
    model.learn(total_timesteps=TIMESTEPS, reset_num_timesteps=False, tb_log_name=f"PPO" )
    print('Iteration ', iters,' has been trained')
    model.save(f"{models_dir}/{TIMESTEPS*iters}")

KeyboardInterrupt: 